In [1]:
import re

In [2]:
def get_csv(row):
    cols = row.split("|")
    #index scan = i_
    #index only scan = i_o_
    #bitmap index scan = b_i_
    #bitmap sequence scan = b_s_
    #sequence scan = s_
    #parallel sequence scan = p_s_
    #order = o_
    #value index = v_i_
    #filter scanned = v_f_
    
    current_sequence = 1
    
    ret_cols = {}
    
    #iterate these in reverse
    
    for col in reversed(cols):
        processed_col = False
    
        if ((col == "") | (col == "\n") | col.startswith("Recheck") | col.startswith("Gather") | col.startswith("BitmapAnd")):
            continue
        
        if(col.startswith("Single Copy: true")):
            ret_cols["forced_parallel"] = 1
            processed_col = True
            
        #cost
        res = re.match(".*Aggregate.*cost=\d*\.\d*..(.*) rows=[0-9].{1}width=[0-9].*\)",col)
        if(res != None):
            ret_cols["cost"] = res.group(1)
            processed_col = True

        #workers
        res = re.match("Workers Planned: (.*)",col)
        if(res != None):
            ret_cols["workers"] = res.group(1) 
            processed_col = True

        #rows
        res = re.match(".*on lineitem  \(cost=.* rows=(.*) width=.*",col)
        if(res != None):
            ret_cols["rows"] = res.group(1)
            processed_col = True

        #single filter scanned
        res = re.match("Filter: \((\w*) [<>] '?([\d\.]+)['::numeric]*?\)",col)
        if (res != None):
            name = res.group(1)
            val = res.group(2)

            ret_cols[normalize_name('v_f_', name)] = val
            ret_cols[normalize_name('s_', name)] = 1
            ret_cols[normalize_name('o_', name)] = current_sequence

            current_sequence += 1
            processed_col = True
            
        #2 filter scanned    
        res = re.match("Filter: \(\((\w*) [<>] '?([\d\.]+)['::numeric]*?\) AND \((\w*) [<>] '?([\d\.]+)['::numeric]*?\)\)",col)
        if (res != None):
            name1 = res.group(1)
            val1 = res.group(2)
            name2 = res.group(3)
            val2 = res.group(4)

            ret_cols[normalize_name('v_f_', name1)] = val1
            ret_cols[normalize_name('s_', name1)] = 1
            ret_cols[normalize_name('o_', name1)] = current_sequence
            current_sequence += 1

            ret_cols[normalize_name('v_f_', name2)] = val2
            ret_cols[normalize_name('s_', name2)] = 1
            ret_cols[normalize_name('o_', name2)] = current_sequence
            current_sequence += 1

            processed_col = True

        #three filter scanned    
        res = re.match("Filter: \(\((\w*) [<>] '?([\d\.]+)['::numeric]*?\) AND \((\w*) [<>] ([\d\.]+)\) AND \((\w*) [<>] ([\d\.]+)\)\)",col)
        if (res != None):
            name1 = res.group(1)
            val1 = res.group(2)
            name2 = res.group(3)
            val2 = res.group(4)
            name3 = res.group(5)
            val3 = res.group(6)

            ret_cols[normalize_name('v_f_', name1)] = val1
            ret_cols[normalize_name('s_', name1)] = 1
            ret_cols[normalize_name('o_', name1)] = current_sequence
            current_sequence += 1

            ret_cols[normalize_name('v_f_', name2)] = val2
            ret_cols[normalize_name('s_', name2)] = 1
            ret_cols[normalize_name('o_', name2)] = current_sequence
            current_sequence += 1

            ret_cols[normalize_name('v_f_', name3)] = val3
            ret_cols[normalize_name('s_', name3)] = 1
            ret_cols[normalize_name('o_', name3)] = current_sequence
            current_sequence += 1

            processed_col = True

        #value index       
        res = re.match("Index Cond: \((.*) [<>] '([\d\.]+)'::numeric\)",col)
        if (res != None):
            name = res.group(1)
            val = res.group(2)

            ret_cols[normalize_name('v_i_', name)] = val
            ret_cols[normalize_name('o_', name)] = current_sequence

            current_sequence  += 1
            processed_col = True
            
        #value index could also look like this
        res = re.match("Index Cond: \((.*) [<>] ([\d\.]+)\)",col)
        if (res != None):
            name = res.group(1)
            val = res.group(2)

            ret_cols[normalize_name('v_i_', name)] = val
            ret_cols[normalize_name('o_', name)] = current_sequence

            current_sequence  += 1
            processed_col = True
            
            

        #index scan
        res = re.match("Index Scan using (.*) on lineitem  \(cost.*?",col)
        if(res != None):
            name = res.group(1) 
            ret_cols[normalize_name('i_', name)] = 1
            ret_cols["type"] = "index"
            processed_col = True

        #index only scan
        res = re.match("Index Only Scan using (.*) on lineitem  \(cost.*?",col)
        if(res != None):
            name = res.group(1) 
            ret_cols[normalize_name('i_o_', name)] = 1
            ret_cols["type"] = "index"
            processed_col = True

        #bitmap index scan
        res = re.match("Bitmap Index Scan on (.*)  \(cost.*?",col)
        if(res != None):
            name = res.group(1) 
            ret_cols[normalize_name('b_i_', name)] = 1
            ret_cols["type"] = "bitmap"
            processed_col = True

        #sequence scan
        res = re.match("Seq Scan on lineitem.*?",col)
        if(res != None):
            ret_cols["type"] = "scan"
            processed_col = True

        #parallel sequence scan
        res = re.match("Parallel Seq Scan on lineitem.*?",col)
        if(res != None):
            ret_cols["type"] = "scan"
            ret_cols["parallel"] = 1
            processed_col = True
            
        if (not processed_col):
            print("I don't know what this column is: " + col)
    
    
    #Now, let's assemble our returned row:
        #index scan = i_
    #index only scan = i_o_
    #bitmap index scan = b_i_
    #sequence scan = s_
    #order = o_
    #value index = v_i_
    #filter scanned = v_f_
    return [
            ret_cols.get("workers", 0),
            ret_cols.get("cost", 0),
        ret_cols.get("rows", 0),
        ret_cols.get("type", 0),
        ret_cols.get("parallel", 0),
        ret_cols.get("i_q", 0),
        ret_cols.get("i_p", 0),
        ret_cols.get("i_s", 0),
        ret_cols.get("i_e", 0),
        ret_cols.get("i_d", 0),
        ret_cols.get("i_o_q", 0),
        ret_cols.get("i_o_p", 0),
        ret_cols.get("i_o_s", 0),
        ret_cols.get("i_o_e", 0),
        ret_cols.get("i_o_d", 0),
        ret_cols.get("b_i_q", 0),
        ret_cols.get("b_i_p", 0),
        ret_cols.get("b_i_s", 0),
        ret_cols.get("b_i_e", 0),
        ret_cols.get("b_i_d", 0),
        ret_cols.get("s_q", 0),
        ret_cols.get("s_p", 0),
        ret_cols.get("s_s", 0),
        ret_cols.get("s_e", 0),
        ret_cols.get("s_d", 0),
        ret_cols.get("o_q", 0),
        ret_cols.get("o_p", 0),
        ret_cols.get("o_s", 0),
        ret_cols.get("o_e", 0),
        ret_cols.get("o_d", 0),
        ret_cols.get("v_i_q", 0),
        ret_cols.get("v_i_p", 0),
        ret_cols.get("v_i_s", 0),
        ret_cols.get("v_i_e", 0),
        ret_cols.get("v_i_d", 0),
        ret_cols.get("v_f_q", 0),
        ret_cols.get("v_f_p", 0),
        ret_cols.get("v_f_s", 0),
        ret_cols.get("v_f_e", 0),
        ret_cols.get("v_f_d", 0),
        ret_cols.get("forced_parallel", 0)
    ]

def normalize_name(pre, name):
    if ((name == 'l_quantity') | (name == 'l_qty')):
        return pre + 'q'
    elif ((name == 'l_extendedprice') | (name == 'l_ep')):
        return pre + 'e'
    elif ((name == 'l_partkey') | (name == 'l_pk')):
        return pre + 'p'
    elif ((name == 'l_suppkey') | (name == 'l_sk')):
        return pre + 's'
    elif ((name == ('l_discount')) | (name == 'l_dis')):
        return pre + 'd'
    
def process_explain(path):
    #remove dashes
    file = open(path, "r") 
    text = file.read() 

    text = re.sub("(-|\\+)", "",text)

    #remove the QUERY PLAN line
    text = re.sub("(.*QUERY PLAN.*\n)", "",text)

    #remove double spacing
    text = re.sub("\n\n", "\n",text)

    #remove spaces at the beginning of the line
    text = re.sub("\n( |-|>)+", "\n",text)

    #replace the \n with |
    text = re.sub("\n", "|",text)

    #replace \\([0-9]+ rows\\) with \n
    text = re.sub("\\([0-9]+ rows\\)", "\n",text)

    #remove superfluous | at beginning and end of text
    text = re.sub("\|\n\|", "\n",text)
    text = re.sub("\n\|", "\n",text)
    
    rows = text.split("\n")
    
    results = [get_csv(t) for t in rows]
    
    df = pd.DataFrame(results, columns=["workers", "cost", "rows", "type", "parallel", "i_q", "i_p","i_s", "i_e", "i_d", "i_o_q", "i_o_p", "i_o_s","i_o_e","i_o_d","b_i_q", "b_i_p", "b_i_s", "b_i_e", "b_i_d", "s_q", "s_p", "s_s", "s_e", "s_d", "o_q", "o_p", "o_s", "o_e", "o_d", "v_i_q", "v_i_p", "v_i_s", "v_i_e", "v_i_d", "v_f_q", "v_f_p", "v_f_s", "v_f_e", "v_f_d","forced_parallel"])
    
    return df

In [552]:
explain_file = "test5/m5/export/explain.txt"
query_file = "test5/m5/export/results.txt"
timing_file = "test5/m5/export/timing.txt"
output_file = "test5/m5/export/test5.txt"

# First, clean up the explain file

In [553]:
file = open(explain_file, "r") 
text = file.read() 

In [554]:
#text

In [555]:
#remove dashes
text = re.sub("(-|\\+)", "",text)

#remove the QUERY PLAN line
text = re.sub("(.*QUERY PLAN.*\n)", "",text)

#remove double spacing
text = re.sub("\n\n", "\n",text)

#remove spaces at the beginning of the line
text = re.sub("\n( |-|>)+", "\n",text)

#replace the \n with |
text = re.sub("\n", "|",text)

#replace \\([0-9]+ rows\\) with \n
text = re.sub("\\([0-9]+ rows\\)", "\n",text)

#remove superfluous | at beginning and end of text
text = re.sub("\|\n\|", "\n",text)
text = re.sub("\n\|", "\n",text)

In [556]:
#text

In [557]:
rows = text.split("\n")

In [558]:
rows[0]

"|Aggregate  (cost=1967864.55..1967864.56 rows=1 width=96)|Seq Scan on lineitem  (cost=0.00..1964795.30 rows=1227698 width=8)|Filter: (l_quantity < '2'::numeric)"

Now, we've got the entire query plan in a row.. one per query

In [559]:
first = rows[0]

In [560]:
first.split("|")

['',
 'Aggregate  (cost=1967864.55..1967864.56 rows=1 width=96)',
 'Seq Scan on lineitem  (cost=0.00..1964795.30 rows=1227698 width=8)',
 "Filter: (l_quantity < '2'::numeric)"]

In [561]:
results = [get_csv(t) for t in rows]

In [562]:
#results

In [563]:
import pandas as pd

In [564]:
df = pd.DataFrame(results, columns=["workers", "cost", "rows", "type", "parallel", "i_q", "i_p","i_s", "i_e", "i_d", "i_o_q", "i_o_p", "i_o_s","i_o_e","i_o_d","b_i_q", "b_i_p", "b_i_s", "b_i_e", "b_i_d", "s_q", "s_p", "s_s", "s_e", "s_d", "o_q", "o_p", "o_s", "o_e", "o_d", "v_i_q", "v_i_p", "v_i_s", "v_i_e", "v_i_d", "v_f_q", "v_f_p", "v_f_s", "v_f_e", "v_f_d","forced_parallel"])

In [565]:
df.head()

,workers,cost,rows,type,parallel,i_q,i_p,i_s,i_e,i_d,...,v_i_p,v_i_s,v_i_e,v_i_d,v_f_q,v_f_p,v_f_s,v_f_e,v_f_d,forced_parallel
0,0,1967864.56,1227698,scan,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
1,0,1971008.78,2485388,scan,0,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,0
2,0,1974118.02,3729082,scan,0,0,0,0,0,0,...,0,0,0,0,4,0,0,0,0,0
3,2,1529057.57,261719,scan,1,0,0,0,0,0,...,0,0,0,0,0,0,1000,0,0,0
4,2,1529173.42,308061,scan,1,0,0,0,0,0,...,0,0,0,0,0,0,1200,0,0,0


In [566]:
df["o_p"].unique()

array([0, 2, 3])

In [567]:
df = process_explain(explain_file)

# The Query File

In [568]:
file = open(query_file, "r") 
text = file.read() 
#text

In [569]:
#remove dashes
text = re.sub("(-|\\+)+\n", "",text)

#replace \\([0-9]+ rows\\) with \n
text = re.sub("\\([0-9]+ row\\)\n", "",text)

#remove spaces
text = re.sub(" ", "",text)

#replace the \n with |
text = re.sub("\|", ",",text)

#remove double spacing
text = re.sub("\n\n", "\n",text)

#remove the header
text = re.sub("selectivity,workers,type,batch,parllelmode,parallelsetupcost,query,i_q,i_p,i_s,i_e,i_d,v_q,v_s,v_d,v_e,v_p\n", "", text)
#text

In [570]:
#read selectivity
rows = text.split("\n")
results = [t.split(",") for t in rows]
df_query = pd.DataFrame(results, columns=["selectivity","workers","type","batch","parllelmode","parallelsetupcost","query","i_q","i_p","i_s","i_e","i_d","v_q","v_s","v_d","v_e","v_p"])
df_query.head()
#df_query = pd.read_csv(query_file)

,selectivity,workers,type,batch,parllelmode,parallelsetupcost,query,i_q,i_p,i_s,i_e,i_d,v_q,v_s,v_d,v_e,v_p
0,0.0200017664106316,0,scan,3-1,0,1000000,2,0,0,0,0,0,2,0,0,0,0
1,0.0399808608841269,0,scan,3-1,0,1000000,2,0,0,0,0,0,3,0,0,0,0
2,0.0600066495457977,0,scan,3-1,0,1000000,2,0,0,0,0,0,4,0,0,0,0
3,0.00998722169613696,2,scan,3-2,1,1000,3,0,0,0,0,0,0,1000,0,0,0
4,0.0119918710436219,2,scan,3-2,1,1000,3,0,0,0,0,0,0,1200,0,0,0


# Timing file

In [586]:
file = open(timing_file, "r") 
text = file.read() 
#text

In [587]:
text = re.sub("You are now connected to database \"tpch\" as user \"postgres\"\.\n", "",text)
text = re.sub("Timing is (off|on)\\.\n", "",text)
text = re.sub("Time:", "",text)
text = re.sub("ms", "",text)
text = re.sub(" ", "",text)
text = re.sub("SET\n", "",text)
text = re.sub("DROPINDEX\n", "",text)
text = re.sub("CREATEINDEX\n", "",text)
text = re.sub("Starting\n", "",text)
text = re.sub("\\(.*\\)", "",text)
#text

In [588]:
#read timing
rows = text.split("\n")
results = [t.split(",") for t in rows]
df_timing = pd.DataFrame(results, columns=["latency"])
df_timing.head()
#df_timing = pd.read_csv(timing_file)

,latency
0,8679.429
1,8767.561
2,8882.784
3,2303.575
4,2316.700


In [589]:
#df_explain = df.drop(df.index[576:677])

In [590]:
#missing = process_explain("test3/m5/export-3bfix/explain.txt")

In [591]:
#missing.head()

In [592]:
#missing.shape

In [593]:
#missing.tail()

In [594]:
#missing = missing[:-1]

In [595]:
#df_explain.iloc[576]

In [596]:
#we need to inser these into the location we took the others out...
#df_explain = pd.concat([df_explain.loc[:576], missing, df_explain.loc[576:]])

In [597]:
#df[(df["v_f_s"] == '75000') & (df["v_f_p"] == '1200000')]

In [598]:
#df_explain[df_explain["v_f_s"] == '5000']

In [599]:
#df_explain[df_explain["v_i_s"] == '5000']
df_explain = df

In [600]:
df_explain.shape, df_timing.shape, df_query.shape

((821, 41), (821, 1), (821, 17))

In [601]:
#df.iloc[576:677]

In [602]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print(df_timing)

In [603]:
#df_query

In [604]:
df_timing[0:2]

,latency
0,8679.429
1,8767.561


In [605]:
#df_timing = df_timing[1:]

In [606]:
df_explain[-5:]

,workers,cost,rows,type,parallel,i_q,i_p,i_s,i_e,i_d,...,v_i_p,v_i_s,v_i_e,v_i_d,v_f_q,v_f_p,v_f_s,v_f_e,v_f_d,forced_parallel
816,2,1653986.40,245563,scan,1,0,0,0,0,0,...,0,0,0,0,0,1700000,75000,89000,0,0
817,2,1653715.86,137350,scan,1,0,0,0,0,0,...,0,0,0,0,0,1200000,64500,89500,0,0
818,2,1528829.98,170683,scan,1,0,0,0,0,0,...,0,0,0,0,0,0,0,95500,0,0
819,2,1528877.13,189545,scan,1,0,0,0,0,0,...,0,0,0,0,0,0,0,94500,0,0
820,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [607]:
df_query[-5:]

,selectivity,workers,type,batch,parllelmode,parallelsetupcost,query,i_q,i_p,i_s,i_e,i_d,v_q,v_s,v_d,v_e,v_p
816,0.00970118853629507,2,scan,3-2,1,1000,14,0,0,0,0,0,0,75000,0,89000,1640000
817,0.00545453466415826,2,scan,3-2,1,1000,15,0,0,0,0,0,0,64000,0,89000,1200000
818,0.00379423203247315,0,scan,4-2,1,1000,6,0,0,0,0,0,0,0,0,95000,0
819,0.0050239178934463,0,scan,4-2,1,1000,6,0,0,0,0,0,0,0,0,95000,0
820,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


The last rows are garbage, so we'll remove them

In [608]:
df_explain = df_explain[:-1]
df_query = df_query[:-1]
df_timing = df_timing[:-1]

In [609]:
df_explain.shape, df_timing.shape, df_query.shape

((820, 41), (820, 1), (820, 17))

In [610]:
df_timing = df_timing.reset_index()
df_explain = df_explain.reset_index()
df_query = df_query.reset_index()

# Combine them and save

In [611]:
df_explain["query"] = df_query["query"]
df_explain["batch"] = df_query["batch"]

In [612]:
df_explain["selectivity"] = df_query["selectivity"]
df_explain["latency"] = df_timing["latency"]

In [613]:
df_explain[df_explain["i_q"] != df_query["i_q"]].head()

,index,workers,cost,rows,type,parallel,i_q,i_p,i_s,i_e,...,v_f_q,v_f_p,v_f_s,v_f_e,v_f_d,forced_parallel,query,batch,selectivity,latency
0,0,0,1967864.56,1227698,scan,0,0,0,0,0,...,2,0,0,0,0,0,2,3-1,0.0200017664106316,8679.429
1,1,0,1971008.78,2485388,scan,0,0,0,0,0,...,3,0,0,0,0,0,2,3-1,0.0399808608841269,8767.561
2,2,0,1974118.02,3729082,scan,0,0,0,0,0,...,4,0,0,0,0,0,2,3-1,0.0600066495457977,8882.784
3,3,2,1529057.57,261719,scan,1,0,0,0,0,...,0,0,1000,0,0,0,3,3-2,0.00998722169613696,2303.575
4,4,2,1529173.42,308061,scan,1,0,0,0,0,...,0,0,1200,0,0,0,3,3-2,0.0119918710436219,2316.700


In [614]:
df_explain = df_explain.drop(columns="index")

In [615]:
df_explain.to_csv(output_file, index=False)

# Combine the A, B and C Files

In [314]:
import pandas as pd
server = "r5"
df_a = pd.read_csv("test3/" + server + "/export-a/test3.txt")
df_b = pd.read_csv("test3/" + server + "/export-b/test3.txt")
df_c = pd.read_csv("test3/" + server + "/export-c/test3.txt")
df_comb = pd.concat([df_a, df_b, df_c])

In [315]:
df_comb.shape

(3700, 45)

In [264]:
df_comb.to_csv("final/r5d4xlarge-test3.txt", index=False)